# Import Libraries

In [ ]:
import gradio as gr
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import warnings
warnings.filterwarnings("ignore")

# Download and Load Model

In [40]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
from facenet_pytorch import MTCNN
mtcnn = MTCNN(keep_all=False, device=DEVICE)


In [41]:
torch.save({
    'model_state_dict': model.state_dict( ),
}, "resnetinceptionv1_epoch_32.pth")

In [42]:
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

checkpoint = torch.load("resnetinceptionv1_epoch_32.pth", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

# Model Inference

In [46]:
def predict(input_image: Image.Image):
    try:
        face = mtcnn(input_image)
        if face is None:
            raise Exception("No face detected")

        face = face.unsqueeze(0)
        face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)

        # Save original face for visualization
        prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().numpy()
        prev_face = (prev_face * 255).astype("uint8")

        face = face.to(DEVICE).float() / 255.0
        face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().numpy()

        target_layers = [model.block8.branch1[-1]]
        use_cuda = torch.cuda.is_available()
        cam = GradCAM(model=model, target_layers=target_layers)
        targets = [ClassifierOutputTarget(0)]

        grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)
        grayscale_cam = grayscale_cam[0, :]
        visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)

        face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)
        face_with_mask = Image.fromarray(face_with_mask)

        with torch.no_grad():
            output = torch.sigmoid(model(face).squeeze(0))
            real_conf = 1 - output.item()
            fake_conf = output.item()
            label = "real" if real_conf > fake_conf else "fake"

        return { "real": real_conf, "fake": fake_conf }, face_with_mask

    except Exception as e:
        print("Prediction Error:", e)
        return { "error": str(e) }, input_image


In [47]:
from PIL import Image
img = Image.open("testing.jpg")  # use a local image path or uploaded one
result, face = predict(img)
print(result)
face.show()

{'real': 0.572949230670929, 'fake': 0.42705076932907104}


# Gradio Interface

In [50]:
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Image(label="Input Image", type="pil") # Remove .inputs from this line
    ],
    outputs=[
        gr.Label(label="Class"), # Remove .outputs from this line
        gr.Image(label="Face with Explainability", type="pil") # Remove .outputs from this line
    ],
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b278fdee8d0b2c4868.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [49]:
torch.save(model, 'final_model.pt')
